In [1]:
%load_ext autoreload
%autoreload 2

from joblib import Parallel, delayed, Memory
import multiprocessing

from tqdm import tqdm, tqdm_notebook
from IPython.display import display
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.patches as patches
%matplotlib inline
import json
import pickle
import os
import sys
sys.path.append('..')
from helper.orderbook_trader import OrderbookTradingSimulator
from helper.manage_orderbooks import *
from helper.evaluation import evaluate, plot_evaluation_costs
from helper.general_helpers import add_features_to_orderbooks, load_and_preprocess_historyfiles

from agents.RL_Agent_Base import RLAgent_Base
from agents.BatchTree_Agent import RLAgent_BatchTree
from agents.QTable_Agent import QTable_Agent

In [2]:
data_nov = pickle.load( open( '../cached_windows_60mins_V200/obs_2016-11_USDT_BTC_maxVol200.p', "rb" ) )
print(len(data_nov))
print(data_nov[0][0])

541
OrderbookContainer from 2016-11-08T10:00 (factor: 1.0)
  190 bids (best: 705.0)
  220 asks (best: 705.450997)
price: 705.2254744655137
  kind: 'orderbook'
  -spread: 0.4509970000000294


In [3]:
data_dec = pickle.load( open( '../cached_windows_60mins/obs_2016-12_USDT_BTC_maxVol100.p', "rb" ) )
print(len(data_dec))
print(data_dec[0][0])

743
OrderbookContainer from 2016-12-01T00:00 (factor: 1.0)
  185 bids (best: 738.1524754)
  247 asks (best: 741.0)
price: 739.5753240682233
  kind: 'orderbook'


In [4]:
data_jan = pickle.load( open( '../cached_windows_60mins/obs_2017-01_USDT_BTC_maxVol100.p', "rb" ) )
print(len(data_jan))
print(data_jan[0][0])

743
OrderbookContainer from 2017-01-01T00:00 (factor: 1.0)
  187 bids (best: 962.86)
  77 asks (best: 964.0)
price: 963.4298875894542
  kind: 'orderbook'


In [5]:
data_feb = pickle.load( open( "../cached_windows_60mins/obs_2017-02_USDT_BTC_maxVol100.p", "rb" ) )
print(len(data_feb))
print(data_feb[0][0])

670
OrderbookContainer from 2017-02-01T00:00 (factor: 1.0)
  139 bids (best: 965.8666755)
  113 asks (best: 965.99)
price: 965.928336437291
  kind: 'orderbook'


In [6]:
data_mar = pickle.load( open( '../cached_windows_60mins_V200/obs_2017-03_USDT_BTC_maxVol200.p', "rb" ) )
print(len(data_mar))
print(data_mar[0][0])

738
OrderbookContainer from 2017-03-01T00:00 (factor: 1.0)
  214 bids (best: 1190.1363916)
  339 asks (best: 1191.2066024)
price: 1190.6714168375438
  kind: 'orderbook'
  -maxSlip_imbalance_disc: 4.0
  -marketPrice_spread_disc: 3.0
  -marketPrice_buy_worst_disc: 2.0
  -spread_disc: 1.0
  -sharecount_imbalance_disc: 1.0
  -direction: 0.9973540893732381
  -direction_disc: -1.0


In [7]:
data_apr = pickle.load( open( '../cached_windows_60mins_V200/obs_2017-04_USDT_BTC_maxVol200.p', "rb" ) )
print(len(data_apr))
print(data_apr[0][0])

719
OrderbookContainer from 2017-04-01T00:00 (factor: 1.0)
  124 bids (best: 1079.95)
  99 asks (best: 1080.0)
price: 1079.9749997962263
  kind: 'orderbook'
  -spread: 0.049999999999954525


In [8]:
data_may = pickle.load( open( '../cached_windows_60mins_V200/obs_2017-05_USDT_BTC_maxVol200.p', "rb" ) )
print(len(data_may))
print(data_may[0][0])

736
OrderbookContainer from 2017-05-01T00:00 (factor: 1.0)
  152 bids (best: 1434.7534716)
  216 asks (best: 1436.0)
price: 1435.3766455891225
  kind: 'orderbook'
  -spread: 1.2465283999999883


In [10]:
def extract_features(window, cash=70000, fixedMarketVariables=False, lim_stepsize=0.1, actions=list(range(-4,11))):
    features = pd.DataFrame()  #[], columns=['_a_{}'.format(a) for a in actions])

    direction = window[-1].get_center() / window[0].get_center()
    for ob in window:
        ts = pd.to_datetime(ob.timestamp)
        
        if fixedMarketVariables:
            # constant features across whole orderbook window
            ob = window[0]
            
        # ob = window[0]
        features.loc[ts, 'ask'] = ob.get_ask()
        features.loc[ts, 'bid'] = ob.get_bid()
        features.loc[ts, 'center'] = ob.get_center()
            
        # level2data
        ob.asks['Volume'] = ob.asks.index * ob.asks.Amount
        ob.asks['accVolume'] = ob.asks.Volume.cumsum()
        ob.bids['Volume'] = ob.bids.index * ob.bids.Amount
        ob.bids['accVolume'] = ob.bids.Volume.cumsum()
        
        for action in actions:
            lim = features.loc[ts, 'ask'] * (1. + (action/100.)*lim_stepsize)

            sub = ob.asks[ob.asks.index <= lim].accVolume
            if len(sub) > 0:
                val = sub.values[-1]
            else:
                sub = ob.bids[ob.bids.index >= lim].accVolume
                if len(sub) > 0:
                    val = -sub.values[-1] 
                else:
                    val = 0
            features.loc[ts, "_a_{}".format(action)] = round(val, 4)

        try:
            features.loc[ts, 'sharecount_buy'], features.loc[ts, 'marketPrice_buy_worst'] = ob.get_current_sharecount(cash=cash)
        except AssertionError:
            features.loc[ts, 'sharecount_buy'] = np.nan
            features.loc[ts, 'marketPrice_buy_worst'] = np.nan
            
        try:
            features.loc[ts, 'sharecount_sell'], features.loc[ts, 'marketPrice_sell_worst'] = ob.get_current_sharecount(cash=-cash)
        except AssertionError:
            features.loc[ts, 'sharecount_sell'] = np.nan
            features.loc[ts, 'marketPrice_sell_worst'] = np.nan

        # features.loc[ts, 'baseVolume'] = hist.loc[ob.timestamp,'baseVolume']
        # features.loc[ts, 'quoteVolume'] = hist.loc[ob.timestamp,'quoteVolume']

        # features['marketPrice_buy'], features['marketPrice_buy_worst'] = ob.get_current_price(volume=shares)
        # features['marketPrice_buy_avg'] = features['marketPrice_buy'] / shares
        # features['marketPrice_sell'], features['marketPrice_sell_worst'] = ob.get_current_price(volume=-shares)
        # features['marketPrice_sell_avg'] = features['marketPrice_sell'] / shares
    direction = window[-1].get_center() / window[0].get_center()
    features['ob_direction'] = direction
    return features

def enhance_features(features, normalize=True, fillna=True, cash=70000):
    if normalize:
        factor = features.center
        features['ask'] /= factor
        features['bid'] /= factor
        features['marketPrice_buy_worst'] /= factor
        features['marketPrice_sell_worst'] /= factor
        features['center_orig'] = features.center
        features['center'] = 1.
    else:
        features['center_orig'] = features.center
        
    if fillna:
        features['sharecount_sell'].interpolate(inplace=True)
        features['sharecount_buy'].interpolate(inplace=True)
        features['marketPrice_sell_worst'].interpolate(inplace=True)
        features['marketPrice_buy_worst'].interpolate(inplace=True)
        
    # enhanced features
    for lookahead in [5, 15, 60]:
        features['future_center{}'.format(lookahead)] = round(features.center_orig.pct_change(lookahead), 4).shift(-lookahead)
    
    # features['volume_imbalance'] = features.baseVolume / features.quoteVolume
    features['orderPriceRatio'] = cash / features.center_orig
    features['spread'] = features.ask - features.bid
    features['sharecount_spread'] = features.sharecount_sell - features.sharecount_buy
    features['sharecount_imbalance'] = (features.sharecount_sell / features.sharecount_buy) - 1.
    features['marketPrice_spread'] = features.marketPrice_sell_worst - features.marketPrice_buy_worst
    features['marketPrice_imbalance'] = abs(features.marketPrice_sell_worst-1.) / (features.marketPrice_buy_worst-1.)
    features['maxSlip_buy'] = features.marketPrice_buy_worst - features.center
    features['maxSlip_sell'] = features.center - features.marketPrice_sell_worst
    features['maxSlip_imbalance'] = features.maxSlip_buy / features.maxSlip_sell
    
    # features = features[sorted(features.columns)]
    
    return features

In [ ]:
data = data_nov + data_dec + data_jan + data_feb + data_mar + data_apr + data_may

In [ ]:
len(data)

In [13]:
data = data_nov + data_dec + data_jan + data_feb + data_mar + data_apr + data_may
cash=70000
num_cores = multiprocessing.cpu_count()
print("len", len(data))
results = Parallel(n_jobs=num_cores, verbose=10)(delayed(extract_features)(
    window=window, 
    cash=cash,
    fixedMarketVariables=True) 
                                                 for window in data)
features = enhance_features(pd.concat(results), normalize=True, cash=cash)
features.to_csv('ob_features70000_1611_1705_level2data_fixed.csv')
print("features:", features.columns)
features

len 4890


[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:   12.7s
[Parallel(n_jobs=24)]: Done  13 tasks      | elapsed:   17.5s
[Parallel(n_jobs=24)]: Done  24 tasks      | elapsed:   22.1s
[Parallel(n_jobs=24)]: Done  37 tasks      | elapsed:   31.2s
[Parallel(n_jobs=24)]: Done  50 tasks      | elapsed:   39.0s
[Parallel(n_jobs=24)]: Done  65 tasks      | elapsed:   47.9s
[Parallel(n_jobs=24)]: Done  80 tasks      | elapsed:   54.9s
[Parallel(n_jobs=24)]: Done  97 tasks      | elapsed:  1.0min
[Parallel(n_jobs=24)]: Done 114 tasks      | elapsed:  1.1min
[Parallel(n_jobs=24)]: Done 133 tasks      | elapsed:  1.2min
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:  1.2min
[Parallel(n_jobs=24)]: Done 173 tasks      | elapsed:  1.3min
[Parallel(n_jobs=24)]: Done 194 tasks      | elapsed:  1.3min
[Parallel(n_jobs=24)]: Done 217 tasks      | elapsed:  1.4min
[Parallel(n_jobs=24)]: Done 240 tasks      | elapsed:  1.5min
[Parallel(n_jobs=24)]: Done 265 tasks      | elapsed:  1.5min
[Paralle

features: Index(['ask', 'bid', 'center', '_a_-4', '_a_-3', '_a_-2', '_a_-1', '_a_0',
       '_a_1', '_a_2', '_a_3', '_a_4', '_a_5', '_a_6', '_a_7', '_a_8', '_a_9',
       '_a_10', 'sharecount_buy', 'marketPrice_buy_worst', 'sharecount_sell',
       'marketPrice_sell_worst', 'ob_direction', 'center_orig',
       'future_center5', 'future_center15', 'future_center60',
       'orderPriceRatio', 'spread', 'sharecount_spread',
       'sharecount_imbalance', 'marketPrice_spread', 'marketPrice_imbalance',
       'maxSlip_buy', 'maxSlip_sell', 'maxSlip_imbalance'],
      dtype='object')


,ask,bid,center,_a_-4,_a_-3,_a_-2,_a_-1,_a_0,_a_1,_a_2,...,future_center60,orderPriceRatio,spread,sharecount_spread,sharecount_imbalance,marketPrice_spread,marketPrice_imbalance,maxSlip_buy,maxSlip_sell,maxSlip_imbalance
2016-11-08 10:00:00,1.000320,0.999680,1.0,-3273.3910,-2336.4171,-111.5535,-111.5535,2047.3466,2237.8183,2730.8499,...,0.0001,99.259035,0.00064,1.866057,0.018974,-0.032865,1.371168,0.013860,0.019005,0.729305
2016-11-08 10:01:00,1.000320,0.999680,1.0,-3273.3910,-2336.4171,-111.5535,-111.5535,2047.3466,2237.8183,2730.8499,...,0.0001,99.259035,0.00064,1.866057,0.018974,-0.032865,1.371168,0.013860,0.019005,0.729305
2016-11-08 10:02:00,1.000320,0.999680,1.0,-3273.3910,-2336.4171,-111.5535,-111.5535,2047.3466,2237.8183,2730.8499,...,0.0001,99.259035,0.00064,1.866057,0.018974,-0.032865,1.371168,0.013860,0.019005,0.729305
2016-11-08 10:03:00,1.000320,0.999680,1.0,-3273.3910,-2336.4171,-111.5535,-111.5535,2047.3466,2237.8183,2730.8499,...,0.0001,99.259035,0.00064,1.866057,0.018974,-0.032865,1.371168,0.013860,0.019005,0.729305
2016-11-08 10:04:00,1.000320,0.999680,1.0,-3273.3910,-2336.4171,-111.5535,-111.5535,2047.3466,2237.8183,2730.8499,...,0.0001,99.259035,0.00064,1.866057,0.018974,-0.032865,1.371168,0.013860,0.019005,0.729305
2016-11-08 10:05:00,1.000320,0.999680,1.0,-3273.3910,-2336.4171,-111.5535,-111.5535,2047.3466,2237.8183,2730.8499,...,0.0001,99.259035,0.00064,1.866057,0.018974,-0.032865,1.371168,0.013860,0.019005,0.729305
2016-11-08 10:06:00,1.000320,0.999680,1.0,-3273.3910,-2336.4171,-111.5535,-111.5535,2047.3466,2237.8183,2730.8499,...,0.0001,99.259035,0.00064,1.866057,0.018974,-0.032865,1.371168,0.013860,0.019005,0.729305
2016-11-08 10:07:00,1.000320,0.999680,1.0,-3273.3910,-2336.4171,-111.5535,-111.5535,2047.3466,2237.8183,2730.8499,...,0.0001,99.259035,0.00064,1.866057,0.018974,-0.032865,1.371168,0.013860,0.019005,0.729305
2016-11-08 10:08:00,1.000320,0.999680,1.0,-3273.3910,-2336.4171,-111.5535,-111.5535,2047.3466,2237.8183,2730.8499,...,0.0001,99.259035,0.00064,1.866057,0.018974,-0.032865,1.371168,0.013860,0.019005,0.729305
2016-11-08 10:09:00,1.000320,0.999680,1.0,-3273.3910,-2336.4171,-111.5535,-111.5535,2047.3466,2237.8183,2730.8499,...,0.0001,99.259035,0.00064,1.866057,0.018974,-0.032865,1.371168,0.013860,0.019005,0.729305


### save features to disk

In [ ]:
display(pd.concat(results).head())

In [ ]:
features = enhance_features(pd.concat(results), normalize=True)
display(features.head())

In [14]:
features.describe()

,ask,bid,center,_a_-4,_a_-3,_a_-2,_a_-1,_a_0,_a_1,_a_2,...,future_center60,orderPriceRatio,spread,sharecount_spread,sharecount_imbalance,marketPrice_spread,marketPrice_imbalance,maxSlip_buy,maxSlip_sell,maxSlip_imbalance
count,293392.000000,293392.000000,293392.0,293392.000000,293392.000000,293392.000000,293392.000000,293392.000000,293392.000000,293392.000000,...,293332.000000,293392.000000,2.933920e+05,293392.000000,293392.000000,293392.000000,293392.000000,293392.000000,293392.000000,293392.000000
mean,1.000904,0.999097,1.0,-17894.192576,-11186.075511,-5599.774794,-1904.441436,3913.022089,11298.324506,18712.797607,...,0.000271,67.888186,1.807160e-03,1.050212,0.014367,-0.024439,-1.194559,0.011455,0.012984,1.414797
std,0.000701,0.000699,0.0,31735.303592,26104.143137,17572.838722,7708.046718,16735.031317,25554.176314,30406.707007,...,0.008812,18.418702,1.400022e-03,0.726086,0.007469,0.013780,361.497606,0.007938,0.008326,14.555445
min,0.999991,0.993720,1.0,-788996.205000,-788996.205000,-788996.205000,-206460.263300,0.000000,0.000000,0.000300,...,-0.135500,26.167078,4.514700e-11,0.045784,0.001411,-0.106982,-18225.414086,-0.000007,0.000004,-0.000811
25%,1.000371,0.998701,1.0,-22954.961500,-13164.545200,-5030.404300,-200.399900,114.121700,2000.949500,4565.747600,...,-0.002100,56.555300,7.413614e-04,0.464636,0.008561,-0.033059,0.779029,0.006175,0.006881,0.598870
50%,1.000803,0.999198,1.0,-8436.740100,-4365.044600,-608.725600,0.000000,1541.902100,5187.364800,10858.764900,...,0.000300,68.033825,1.605253e-03,0.842648,0.012739,-0.020563,1.133367,0.009274,0.010733,0.881167
75%,1.001300,0.999629,1.0,-3290.938700,-446.683900,0.000000,0.000000,3453.814000,12265.202300,22508.315800,...,0.003000,80.958814,2.599092e-03,1.537510,0.018952,-0.013615,1.668757,0.014610,0.017314,1.282960
max,1.006307,1.000015,1.0,0.000000,0.000000,0.000000,0.000000,617513.479800,617513.479800,617513.479800,...,0.104700,101.850446,1.258714e-02,5.564047,0.065371,-0.002614,9303.380609,0.080754,0.074793,844.590633


### plot_episode(data_nov[116])

In [ ]:
features = pd.read_csv('ob_features70000_1611_1705.csv', index_col=0, parse_dates=[0])

In [ ]:
features.head()

In [ ]:
fig = plt.figure(figsize=(12,5))
ax = plt.subplot()
features.center_orig.plot(label="USDT/BTC", ax=ax)
ax.set_ylabel("Price in $")
plt.legend(loc='best')
ax2 = ax.twinx()
ax2.set_ylabel("Relative market order slippage")

features.marketPrice_imbalance.plot(color='green', label='marketPrice_imbalance', ax=ax2, alpha=0.2)
ax2.set_ylim((-10, 10))
plt.legend(loc='best')
plt.title("Historic Prices for currency pair 'USDT/BTC'")
plt.tight_layout()
plt.show()
# plt.savefig("bitcoin_historicPrices.pdf")

In [ ]:
features.center_orig.multiply(features.marketPrice_sell_worst).head()

In [ ]:
features.sharecount_buy.head()

In [ ]:
fig = plt.figure(figsize=(12,5))
ax = plt.subplot()
features.center_orig.plot(label="USDT/BTC", ax=ax)
#features.center_orig.multiply(features.marketPrice_sell_worst).plot(label='sell', ax=ax)
ax.set_ylabel("Price in $")
plt.legend(loc='upper left')
ax2 = ax.twinx()
ax2.set_ylabel("Relative market order slippage for a 70.000$ trade")

(70000. / features.sharecount_buy).divide(features.center_orig).rolling( 240).mean().plot(color='red', label='buy slippage (running mean: 240min)', ax=ax2, alpha=0.2)
(70000. / features.sharecount_sell).divide(features.center_orig).rolling(240).mean().plot(color='green', label='sell slippage (running mean: 240min)', ax=ax2, alpha=0.2)
ax2.grid(False)
#features.marketPrice_buy_worst.rolling( 60).mean().plot(color='red', label='buy slippage (running mean: 240min)', ax=ax2, alpha=0.2)
#features.marketPrice_sell_worst.rolling(60).mean().plot(color='green', label='sell slippage (running mean: 240min)', ax=ax2, alpha=0.2)
plt.axhline(1, alpha=0.3)
plt.legend(loc='lower right')
plt.title("Historic Prices for currency pair 'USDT/BTC'")
plt.tight_layout()
plt.savefig("bitcoin_historicPrices.pdf")

In [ ]:
features.columns

In [ ]:
features['ob_direction_disc'] = pd.cut(features.ob_direction, bins=[-np.inf, 0.996, 0.998, 1.002, 1.004, np.inf], labels=False)
features['future_center60_disc'] = pd.cut(features.future_center60, bins=[-np.inf, -0.002, -0.001, 0.001, 0.002, np.inf], labels=False)
features['future_center60_disc'] = features['future_center60_disc'].dropna().astype(int)
display(features.future_center60.describe())
display(features.future_center60_disc.describe())

In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA


def pca_and_plot(df, cols_x, col_y):
    X = df[cols_x].dropna().copy()
    pca = PCA(n_components=3)
    xhat = pd.DataFrame(pca.fit_transform(X))
    
    y = df.loc[X.index, col_y].astype(int)
    
    
    X = pca.fit_transform(X)
    

    # for name, label in [('Setosa', 0), ('Versicolour', 1), ('Virginica', 2)]:
    #     ax.text3D(X[y == label, 0].mean(),
    #               X[y == label, 1].mean() + 1.5,
    #               X[y == label, 2].mean(), name,
    #               horizontalalignment='center',
    #               bbox=dict(alpha=.5, edgecolor='w', facecolor='w'))
    # Reorder the labels to have colors matching the cluster results
    y = np.choose(y, [0, 1, 2, 3, 4]).astype(np.float)
    
    fig = plt.figure(1, figsize=(16, 12))
    plt.clf()
    ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
    ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=y, cmap=plt.cm.spectral)

    # ax.w_xaxis.set_ticklabels([])
    # ax.w_yaxis.set_ticklabels([])
    # ax.w_zaxis.set_ticklabels([])

plt.show()

   #  colors = sns.color_palette("muted")
   #  xhat['color'] = [colors[int(y)] for y in Y]
   #  
   #  display(xhat.head())
   #  display(xhat.color[0])
   #  
   #  xhat.plot.scatter(x=0, y=1, c=xhat['color'])

cols = list(features.columns)
for elem in ['ob_direction', 'future_center5', 'future_center15']:
    cols.remove(elem)

    
    
#cols = ['spread', 'sharecount_spread', 'marketPrice_spread']
print(cols)
pca_and_plot(df=features.dropna(),
             cols_x=cols,
             col_y='future_center60_disc')

In [ ]:
Y.unique()
colors = sns.color_palette("muted")
Y_color = [colors[y] for int(y) in Y]

In [ ]:
import seaborn as sns

plt.scatter(Xhat[:,0], Xhat[:,1], color=Y_color)

In [ ]:
def plot_orderbook_window(window):
    f = plt.figure(figsize=(12, 8))
    f.suptitle("Orderbook Visualization for cash: {}".format(cash))
    gs = gridspec.GridSpec(3, 2, height_ratios=[1, 1, 1])
    ax1 = f.add_subplot(gs[0,0])
    ax2 = f.add_subplot(gs[1,0], sharex=ax1)
    ax3 = f.add_subplot(gs[2,0], sharex=ax1)

    ax1b = f.add_subplot(gs[0,1])
    ax2b = f.add_subplot(gs[1,1], sharex=ax1b)
    ax3b = f.add_subplot(gs[2,1], sharex=ax1b)

    # plot
    price_vars = ['ask', 'center', 'bid', 'marketPrice_buy_worst', 'marketPrice_sell_worst']
    colors = ['red', 'grey', 'green', 'red', 'green']
    window[price_vars].multiply(window.center_orig, axis="index").plot(ax=ax1, color=colors)
    ax1.set_ylabel("price")
    window[price_vars].plot(ax=ax2, color=colors)
    ax2.set_ylabel("norm price")
    window[['sharecount_buy', 'sharecount_sell']].plot(ax=ax3, rot=65, color=['red', 'green'])
    ax3.set_ylabel("share count")

    vars_slippage = ['maxSlip_buy', 'maxSlip_sell']
    window[vars_slippage].multiply(window.center_orig, axis="index").plot(ax=ax1b, color=['red', 'green'])
    window[vars_slippage].plot(ax=ax2b, color=['red', 'green'])

    window['sharecount_spread'].plot(ax=ax3b, rot=65)
    
ob_idx = 18
plot_orderbook_window(window=features[ob_idx*60:(ob_idx+1)*60])

In [ ]:
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [ ]:
[-np.inf] + list(quantiles.values) + [np.inf]

In [ ]:
quantiles = features.spread.quantile([1/3, 2/3])
display(quantiles)
features['spread_disc'] = pd.cut(features.spread.values, bins=[-np.inf] + list(quantiles.values) + [np.inf], labels=False)
features[['spread', 'spread_disc']]

In [ ]:
df = features
sns.regplot(df.future_center15, df.spread_disc)

In [ ]:
sns.regplot(df.future_center15, df.spread)

In [ ]:
df.columns

In [ ]:
features_of_interest = ['marketPrice_buy_worst',
       'marketPrice_sell_worst', 'marketPrice_spread', 'maxSlip_buy',
       'maxSlip_imbalance', 'maxSlip_sell', 'ob_direction', 'quoteVolume',
       'sharecount_buy', 'sharecount_imbalance', 'sharecount_sell',
       'sharecount_spread', 'spread', 'volume_imbalance', 'spread_disc']

In [ ]:
df = features
for feat in features_of_interest:
    print(feat)
    sns.regplot(df.future_center15, df[feat])
    plt.title(feat)
    plt.show()

##### hist.loc['2016-11-08 10:00:00',:]

In [ ]:
data = data_feb
ob = data[23][48]
display(ob.head())
display(ob.get_current_sharecount(cash=70000))
display(ob.get_current_sharecount(cash=-70000))

In [ ]:
plot_episode(data[0], kind='avg')

In [ ]:
features